In [2]:
import json
dictionary={}
hateful_or_not={}
count=0
with open ("../data/train.jsonl",'r') as f:
    for line in f:
        elem=json.loads(line)
        if (int(elem['label'])==0):
            hateful_or_not[int(elem['id'])]=0
        else:
            hateful_or_not[int(elem['id'])]=1
            count+=1
# print(count)
probab_hateful=count/len(hateful_or_not)
words_captions_hateful={}
words_captions_normal={}
with open("caption.json", 'r') as f:
    dictionary=json.load(f)
    for (id,sentence) in dictionary.items():
        # print(sentence)
        for word in sentence['sentence'].split(" "):
            if (hateful_or_not[int(id)]==0):
                if word in words_captions_normal:
                    words_captions_normal[word]+=1
                else:
                    words_captions_normal[word]=1
            else:
                if word in words_captions_hateful:
                    words_captions_hateful[word]+=1
                else:
                    words_captions_hateful[word]=1


print(words_captions_hateful)
print(words_captions_normal)
with open("words_captions_hateful.json", 'w') as f:
    json.dump(words_captions_hateful,f)
with open("words_captions_normal.json", 'w') as f:
    json.dump(words_captions_normal,f)

{'a': 6817, 'black': 325, 'and': 1254, 'white': 319, 'photo': 160, 'of': 1365, 'man': 1325, 'in': 1470, 'uniform': 62, 'with': 1680, 'mustache': 79, 'arafed': 365, 'turban': 44, 'microphone': 101, 'image': 163, 'hugging': 13, 'another': 19, 'suit': 214, 'arafing': 1, 'beats': 1, 'like': 13, 'the': 608, 'twin': 1, 'towers': 1, 'there': 1920, 'is': 1942, 'picture': 313, 'city': 13, 'red': 88, 'circle': 5, 'cartoon': 28, 'monkey': 21, 'riding': 18, 'horse': 6, 'on': 788, 'it': 176, 'yellow': 16, 'hat': 118, 'glasses': 43, 'smiling': 128, 'police': 23, 'officer': 7, 'walking': 47, 'dog': 102, 'behind': 15, 'tape': 4, 'driving': 9, 'car': 59, 'caption': 366, 'sitting': 391, 'at': 240, 'table': 102, 'pizza': 6, 'knife': 11, 'hand': 101, 'fist': 17, 'that': 681, 'pointing': 48, 'camera': 72, 'shirt': 130, 'woods': 6, 'sign': 94, 'says': 62, 'walmart': 1, 'supercentr': 1, 'they': 70, 'are': 648, 'two': 318, 'men': 126, 'standing': 358, 'next': 87, 'to': 199, 'each': 96, 'other': 100, 'middle':

In [28]:
import numpy as np

class NaiveBayesClassifier:
    def __init__(self):
        self.class_prior = {}
        self.word_likelihoods = {}

    def train(self, word_counts, hateful_dict, normal_dict):
        # for val in hatef
        total_hateful=0
        total_normal=0
        for val in hateful_dict.values():
            total_hateful+=val
        for val in normal_dict.values():
            total_normal+=val
        # total_hateful = sum(hateful_counts)
        # total_normal = sum(normal_counts)
        total_examples = total_hateful + total_normal
        
        self.class_prior[1] = total_hateful / total_examples
        self.class_prior[0] = total_normal / total_examples
        # self.class_prior[1] = probab_hateful
        # self.class_prior[0] = 1 - probab_hateful
        
        for word in word_counts:
            hateful_count=0
            normal_count=0
            if (word in hateful_dict):
                hateful_count=int(hateful_dict[word])
            if (word in normal_dict):
                normal_count=int(normal_dict[word])
            prob_hateful = (hateful_count + 1) / (total_hateful + len(word_counts))
            prob_normal = (normal_count + 1) / (total_normal + len(word_counts))
            self.word_likelihoods[word] = (prob_hateful, prob_normal)

    def predict(self, X_test):
        predictions = []
        for text in X_test:
            prob_hateful = np.log(self.class_prior[1])
            prob_normal = np.log(self.class_prior[0])
            for word, count in text.items():
                if word in self.word_likelihoods:
                    prob_hateful += count * np.log(self.word_likelihoods[word][0])
                    prob_normal += count * np.log(self.word_likelihoods[word][1])
                    print(prob_hateful)
                    print(prob_normal)
            predictions.append(1 if prob_hateful > prob_normal else 0)
        return predictions

# Example usage:
# print(prob_hateful)
# word_counts = ["love", "hate", "awesome", "stupid"]
# hateful_counts = [1, 3, 0, 1]
# normal_counts = [2, 0, 1, 2]
word_counts=[]
for word,count in words_captions_hateful.items():
    word_counts.append(word)
for word,count in words_captions_normal.items():
    if word not in word_counts:
        word_counts.append(word)


classifier = NaiveBayesClassifier()
classifier.train(word_counts,words_captions_hateful, words_captions_normal)

X_test = [{"black": 5}, {"bhai": 1, "everyone": 1}]
predictions = classifier.predict(X_test)
print(predictions)  # Output: [1, 0] indicating hateful and non-hateful respectively


-25.258391693818485
-26.635704879360567
-10.962371918993192
-10.235624990757906
[1, 0]


In [13]:
import numpy as np

class GaussianNaiveBayesClassifier:
    def __init__(self):
        self.class_prior = {}
        self.word_stats = {}

    def train(self, word_counts, hateful_dict, normal_dict):
        total_hateful=0
        total_normal=0
        for val in hateful_dict.values():
            total_hateful+=val
        for val in normal_dict.values():
            total_normal+=val
        # self.class_prior[1] = probab_hateful
        # self.class_prior[0] = 1 - probab_hateful
        
        self.class_prior[1] = total_hateful / (total_hateful+total_normal)
        self.class_prior[0] = total_normal / (total_hateful+total_normal)
        
        for word in word_counts:
            hateful_count=0
            normal_count=0
            if (word in hateful_dict):
                hateful_count=int(hateful_dict[word])
            if (word in normal_dict):
                normal_count=int(normal_dict[word])
            mean_hateful = hateful_count / total_hateful
            mean_normal = normal_count / total_normal
            variance_hateful = (hateful_count * (1 - mean_hateful)**2 + (total_hateful - hateful_count) * (0 - mean_hateful)**2) / total_hateful
            variance_normal = (normal_count * (1 - mean_normal)**2 + (total_normal - normal_count) * (0 - mean_normal)**2) / total_normal
            self.word_stats[word] = (mean_hateful, variance_hateful, mean_normal, variance_normal)

    def predict(self, X_test):
        predictions = []
        for text in X_test:
            prob_hateful = np.log(self.class_prior[1])
            prob_normal = np.log(self.class_prior[0])
            for word, count in text.items():
                if word in self.word_stats:
                    mean_hateful, variance_hateful, mean_normal, variance_normal = self.word_stats[word]
                    prob_hateful += self.gaussian_prob(count, mean_hateful, variance_hateful)
                    prob_normal += self.gaussian_prob(count, mean_normal, variance_normal)
            predictions.append(1 if prob_hateful > prob_normal else 0)
        return predictions


    def gaussian_prob(self, x, mean, variance):
        exponent = -((x - mean) ** 2) / (2 * (variance+1e-9))
        return np.exp(exponent) / np.sqrt(2 * np.pi * (variance+1e-9))

# Example usage:
# word_counts = ["love", "hate", "awesome", "stupid"]
# hateful_counts = [1, 3, 0, 1]
# normal_counts = [2, 0, 1, 2]

classifier = GaussianNaiveBayesClassifier()
classifier.train(word_counts, words_captions_hateful, words_captions_normal)

X_test = [{"black": 50, "dark": 100}, {"hate": 1, "everyone": 1}]
predictions = classifier.predict(X_test)
print(predictions)  # Output: [1, 0] indicating hateful and non-hateful respectively


[0, 0]
